In [ ]:
%load_ext autoreload
%autoreload 2

from discontinuum.providers import usgs
from discontinuum.utils import aggregate_to_daily

import xarray as xr

In [ ]:
# configuration
# Choptank River at Greensboro, MD
#site = "01491000" 
#start_date = "1979-10-01"
#end_date = "2011-09-30"
#target_pcode = "00631" #5 digit USGS code

# SF Coeur D Alene River 
site = "12413470"
#parameter_cd = "00665"
parameter_cd = "00631"
start= "1988-10-01" 
end= "2021-09-30" 

In [ ]:
# download data

daily = usgs.get_daily(site=site, start_date=start_date, end_date=end_date)
samples = usgs.get_samples(site=site, start_date=start_date, end_date=end_date, pcode=target_pcode)

samples = aggregate_to_daily(samples)

training_data = xr.merge([samples, daily], join='inner')

In [ ]:
# fit model

from discontinuum.models import WRTDSModel

model = WRTDSModel()
model.fit(target=training_data['concentration'],
          covariates=training_data[['time','flow']])

In [ ]:
# plot result

model.plot_concentration(daily[['time','flow']])